# 03 angr symbolic registers

First we load the compiled binary executable:

In [43]:
import angr, claripy, sys

libname = "../build/arm64-v8a/03_angr_symbolic_registers"
project = angr.Project(libname, main_opts={ "base_addr": 0x00100000 })
print(project.arch, hex(project.entry))

<Arch AARCH64 (LE)> 0x10070c


We want start our execetion after invoke `get_user_input` function:(e.g.: 0x00101254)

In [44]:
after_get_user_input_address = int(input("after get_user_input address(hex): "), 16)

We prepare the **blank state** for symbolic execution:

In [45]:
blank_state = project.factory.blank_state(addr=after_get_user_input_address)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_MEMORY)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_REGISTERS)

Let's create some symbolic registers which return as result by `get_user_input` function:
```
        002011f0 dc ff ff 97     bl         get_user_input
        002011f4 e5 0f 00 b9     str        w5,[sp, #local_24]
        002011f8 e6 0b 00 b9     str        w6,[sp, #local_28]
        002011fc e7 07 00 b9     str        w7,[sp, #local_2c]
```

In [46]:
w5 = claripy.BVS('w5', 32)
w6 = claripy.BVS('w6', 32)
w7 = claripy.BVS('w7', 32)

blank_state.regs.w5 = w5
blank_state.regs.w6 = w6
blank_state.regs.w7 = w7

Everything is ready, next we start the symbolic execution to find the path:

In [47]:
def is_successful(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Good Job.".encode() in stdout_output 

def should_abort(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Try again.".encode() in stdout_output

simulation = project.factory.simulation_manager(blank_state)
simulation.explore(find=is_successful, avoid=should_abort)
if simulation.found:
    solution_w5 = simulation.found[0].solver.eval(w5)
    solution_w6 = simulation.found[0].solver.eval(w6)
    solution_w7 = simulation.found[0].solver.eval(w7)

    solution = ' '.join(map('{:x}'.format, [ solution_w5, solution_w6, solution_w7 ])) 
    print("Solution(password):", solution)
else:
    print("Cound not find the solution")
    list(map(lambda s: s.callstack.dbg_repr(), simulation.deadended))

Solution(password): 62f0141a 4b6e02bf 1b503e8f
